In [2]:
!pip install tweepy
import pandas as pd
import datetime
import sys
import tweepy
import re
import json

  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)


You should consider upgrading via the 'c:\users\daniel\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [17]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r'', string)

In [18]:
def obter_tweets(usuario, limite=10):
    df_tweets = pd.DataFrame(columns=['dat_tweet','user_tweet','tweet'])
    resultados = api.user_timeline(screen_name=usuario, count=limite, tweet_mode='extended')
    
    for r in resultados:
        tweet = re.sub(r'http\S+', '', r.full_text)
        tweet = tweet.replace('\n', ' ')
        tweet = remove_emoji(tweet)
        dat_tweet = str(r.created_at)
        user_tweet = usuario

        new_row = {'dat_tweet':dat_tweet, 'user_tweet':user_tweet, 'tweet':tweet}
        df_tweets = df_tweets.append(new_row, ignore_index=True)

    return df_tweets 

In [19]:
def obter_tweets_por_data(usuario):

    df_tweets = pd.DataFrame(columns=['dat_tweet','user_tweet','tweet'])
    startDate = datetime.datetime(2021, 1, 1, 0, 0, 0)
    endDate =   datetime.datetime(2021, 12, 31, 0, 0, 0)
    tweets = []
  
    tmpTweets = api.user_timeline(screen_name= usuario, tweet_mode='extended')
    for tweet in tmpTweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            tweets.append(tweet)

    while (tmpTweets[-1].created_at > startDate):
        print("Last Tweet @", tmpTweets[-1].created_at, " - fetching some more")
        tmpTweets = api.user_timeline(usuario, max_id = tmpTweets[-1].id)
        for tweet in tmpTweets:
            if tweet.created_at < endDate and tweet.created_at > startDate:
                tweets.append(tweet)
  
    for tweet in tweets:
        msg = re.sub(r'http\S+', '', tweet.full_text)
        msg = msg.replace('\n', ' ')
        msg = remove_emoji(msg)
        dat_tweet = str(tweet.created_at)
        user_tweet = usuario

        new_row = {'dat_tweet':dat_tweet, 'user_tweet':user_tweet, 'tweet':msg}
        df_tweets = df_tweets.append(new_row, ignore_index=True)

    print("DataFrame from @{} ready!".format(user_tweet))
    return(df_tweets)

In [25]:
f = open('credentials.json',)
data = json.load(f)
f.close()

consumer_key = data['credentials'][0]['consumer_key']
consumer_secret = data['credentials'][0]['consumer_secret']
access_token = data['credentials'][0]['access_token']
access_token_secret = data['credentials'][0]['access_token_secret']

# acessar https://apps.twitter.com para criar uma nova aplicação
# cada aplicação tem suas próprias chaves
# acessar a aba "Keys and Access Tokens"
# passa o Consumer Key e o Consumer Secret
auth = tweepy.OAuthHandler(consumer_key, 
                           consumer_secret)

# define o token de acesso
# para criar basta clicar em "Create my access token"
# passa o "Access Token" e o "Access Token Secret"
auth.set_access_token(access_token,
                      access_token_secret)

api = tweepy.API(auth)

usuarios = ['infomoney', 'EstadaoEconomia', 'UOLEconomia', 'InvestingBrasil', 'leiamoneytimes']
#usuarios = ['infomoney']
df = pd.DataFrame(columns=['dat_tweet','user_tweet','tweet'])
for usuario in usuarios:
    df = df.append(obter_tweets(usuario=usuario, limite=100))
    #df = df.append(obter_tweets_por_data(usuario=usuario))

In [28]:
df.sort_values(by=['user_tweet', 'dat_tweet'])
df.head(100)

,dat_tweet,user_tweet,tweet
0,2021-04-14 15:10:35,infomoney,RT @AlexSchwartsman: Um polímata às avessas M...
1,2021-04-14 14:55:52,infomoney,Bitcoin chega à bolsa dos EUA com estreia da C...
2,2021-04-14 14:27:10,infomoney,"Bernard Madoff, criador da maior fraude financ..."
3,2021-04-14 13:58:44,infomoney,Brasil registra 3.808 novas mortes por Covid-1...
4,2021-04-14 13:23:20,infomoney,Magazine Luiza anuncia acordo para aquisição d...
...,...,...,...
95,2021-04-12 12:13:19,infomoney,Bancos devem voltar a estender prazo de dívida...
96,2021-04-12 11:58:52,infomoney,Petrobras elegerá novos conselheiros em assemb...
97,2021-04-12 11:27:58,infomoney,Embaixador dos EUA afirma que Brasil terá de p...
98,2021-04-12 10:56:33,infomoney,Os 5 assuntos que vão movimentar o mercado nes...


In [27]:
df.shape

(500, 3)